### This notebook is to generate df of Positions, that is resulted in our strategy

In [25]:
import pandas as pd
from math import sqrt
import math

import requests
import json
import numpy as np

import UNI_v3_funcs as liq_amounts
import load_data
import collections

### Query Functions

In [2]:
def get_cprice_tick_at_mint(token0, token1, feeTier, mint_time, decimal0, decimal1):
    
    # get token_ids
    token0_id = load_data.get_token_id(token0)
    token1_id = load_data.get_token_id(token1)
    pool_id = load_data.get_pool_id(token0_id, token1_id, feeTier)
        
    # query price at that specific time
    query_ = """
    {{
      poolHourDatas(first:1,
      where:{{ pool: "{}",
      periodStartUnix: {} }}
      ){{
        tick
      }}
    }}""".format(pool_id, mint_time)
    
    query_result_ = load_data.run_query(query_)
    json_data_ = json.loads(query_result_.text)
    
    ctick = int(json_data_['data']['poolHourDatas'][0]['tick'])
    cprice = tick_to_price(ctick,decimal0, decimal1)
    
    return cprice, ctick

def get_decimals(token_symbol):
    query_ = """
    {{
      tokens(first:10,
      where:{{ symbol: "{}"}}
      ){{
        decimals
      }}
    }}""".format(token_symbol)
    
    query_result_ = load_data.run_query(query_)
    json_data_ = json.loads(query_result_.text)
    decimal_ = float(json_data_['data']['tokens'][0]['decimals'])
    
    return decimal_

### Positions Genrations

In [19]:
class BlockPosition:
    def __init__(self, starting_tick, block_size, block_spacing, block_amount):
        self.block_size = block_size
        self.block_spacing = block_spacing
        self.block_amount = block_amount
        self.k = (block_amount-1)/2
        self.center_tick = starting_tick
        self.next_pos_index = 1
        self.block_array = self.gen_block_array()
        
    def gen_block_array(self):
        block_array = collections.deque()
        first_tick = int(self.center_tick - self.k * self.block_spacing)
        for i in range(self.block_amount):
            curr_tick = int(first_tick + i*self.block_spacing)
            block_array.append({ 
                'index': self.next_pos_index,
                'lower_tick': curr_tick - int(self.block_size/2),
                'upper_tick': curr_tick - int(self.block_size/2) + self.block_size - 1
            })
            self.next_pos_index = self.next_pos_index + 1
        return block_array
    
    def shift_left(self):
        self.center_tick -= self.block_spacing
        prev_lower_tick = self.block_array[0]['lower_tick']
        prev_upper_tick = self.block_array[0]['upper_tick']
        
        self.block_array.appendleft({ 
            'index': self.next_pos_index,
            'lower_tick': prev_lower_tick - self.block_spacing,
            'upper_tick': prev_upper_tick - self.block_spacing
        })
        self.block_array.pop()

        self.next_pos_index += 1
 
    def shift_right(self):
        self.center_tick += self.block_spacing
        prev_lower_tick = self.block_array[-1]['lower_tick']
        prev_upper_tick = self.block_array[-1]['upper_tick']
        
        self.block_array.append({ 
            'index': self.next_pos_index,
            'lower_tick': prev_lower_tick + self.block_spacing,
            'upper_tick': prev_upper_tick + self.block_spacing
        })
        self.block_array.popleft()

        self.next_pos_index += 1




df_merged = df_merged.sort_values('periodStartUnix', ignore_index = True)


In [3]:
def get_positions_fixed_strategy(mint_time, burn_time, upper_, lower_, 
                                 decimal0_, decimal1_, feetier_, 
                                 mint_amount0, mint_amount1, mint_tick):
    
    df_positions = pd.DataFrame()
    df_positions['periodStartUnix'] = list(range(mint_time, burn_time+1, 3600))
    df_positions['upper'] = upper_
    df_positions['lower'] = lower_
    df_positions['upper_tick'] = price_to_tick(upper_, decimal0_, decimal1_)
    df_positions['lower_tick'] = price_to_tick(lower_, decimal0_, decimal1_)
    
    df_positions['mint_amount0'] = mint_amount0
    df_positions['mint_amount1'] = mint_amount1
    df_positions['decimal0'] = decimal0_
    df_positions['decimal1'] = decimal1_
    df_positions['feeTier'] = int(feetier_)    
    df_positions['mint_tick'] = int(mint_tick)
    
    return df_positions

In [20]:
def convert_token0_to_1(tick, token1):
    P = 1.0001**tick
    return token1*P
    
def get_amounts(targetAmount, tick, tick_lower, tick_upper):
    Pl = 1.0001**tick_lower
    P = 1.0001**tick
    Pu = 1.0001**tick_upper
    
    if P >= Pu:
        return (0, targetAmount)
    elif Pl <= P and P < Pu:
        deltaL = targetAmount / ((sqrt(P) - sqrt(Pl)) + (1 / sqrt(P) - 1 / sqrt(Pu)) * P)
        y = deltaL * (1 / sqrt(P) - 1 / sqrt(Pu))
        x = deltaL * (sqrt(P) - sqrt(Pl))
        return (y, x)
    elif Pl > P:
        return (targetAmount/P, 0)


In [34]:
def get_positions_dynamic_strategy(df_merged, capital_per_block, block_amount, block_size, block_spacing, shift_theta, decimal0_, decimal1_, feetier_):

    pos = BlockPosition(df_merged.iloc[0]['tick'], block_size, block_spacing, block_amount)
    pos_array = []

    for index, row in df_merged.iterrows():
    

        curr_tick = row.tick
        # if(put_in_price_tick != 0):
        #     print(hourly_fee * put_in_price_tick/tvl_per_tick)
        optimized = False
        changed = False

        if(abs(pos.center_tick - curr_tick) > pos.block_amount * pos.block_spacing):
            pos.center_tick = curr_tick
            changed = True
            pos.block_array = pos.gen_block_array()
        else:
            while(True):
                if(abs(pos.center_tick - curr_tick) >= shift_theta):
                    changed = True
                    if(pos.center_tick > curr_tick):
                        pos.shift_left()
                    else:
                        pos.shift_right()
                else:
                    break

        array_item = {}
        
        array_item['decimal0'] = decimal0_
        array_item['decimal1'] = decimal1_
        array_item['periodStartUnix'] = row['periodStartUnix']
        array_item['feeTier'] = int(feetier_)    
        array_item['mint_tick'] = int(row['tick'])
        
        for idx, block in enumerate(pos.block_array):
            array_item['index_'+str(idx)] = block['index']
            array_item['lower_tick_'+str(idx)] = block['lower_tick']
            array_item['upper_tick_'+str(idx)] = block['upper_tick']
            amount0, amount1 = get_amounts(convert_token0_to_1(row['tick'], capital_per_block), row['tick'], block['lower_tick'], block['upper_tick'])
            array_item['mint_amount0_' + str(idx)] = amount0
            array_item['mint_amount1_' + str(idx)] = amount1
            
        array_item['periodStartUnix'] = row['periodStartUnix']
        pos_array.append(array_item)

    pos_df = pd.DataFrame(pos_array)

    return pos_df

### Calculations Functions

In [4]:
def price_to_tick(price_, decimal0_, decimal1_):
    # convert price (token1 by token0) to tick
    tick_ = math.floor( math.log(price_*10**(decimal1_-decimal0_), 1.0001) )
    return tick_

def tick_to_price(tick_,decimal0_, decimal1_):
    price_ = (1.0001**tick_/10**(decimal1_-decimal0_))
    return price_

In [5]:
def check_inRange(ctick, lower_tick, upper_tick):

    if int(ctick) >= int(upper_tick): # change from > to >=
        inRange = 0
    elif int(ctick) < int(lower_tick):
        inRange = 0
    else: 
        inRange = 1
    
    return inRange

In [6]:
def convert_token0_to_1(tick, token1):
    P = 1.0001**tick
    return token1*P
    
def get_amounts(targetAmount, tick, tick_lower, tick_upper):
    Pl = 1.0001**tick_lower
    P = 1.0001**tick
    Pu = 1.0001**tick_upper
    
    if P >= Pu:
        return (0, targetAmount)
    elif Pl <= P and P < Pu:
        deltaL = targetAmount / ((sqrt(P) - sqrt(Pl)) + (1 / sqrt(P) - 1 / sqrt(Pu)) * P)
        y = deltaL * (1 / sqrt(P) - 1 / sqrt(Pu))
        x = deltaL * (sqrt(P) - sqrt(Pl))
        return (y, x)
    elif Pl > P:
        return (targetAmount/P, 0)

In [7]:
# # Calculate Fees
# # TODO: Clean this up (make it DF operations, make it just 1 function)


# def get_fees(lower_tick_, upper_tick_, ctick_,  L_Pool_, swap_Volume_, amount0_p, amount1_p, amt0_, amt1_, decimal0_, decimal1_, feeTier, mint_tick):
#     # (lower_tick_, upper_tick_) from df_positions
#     # (ctick, L_Pool, swap_volume, amount0_p, amount1_p) from df_merged
#     # (amt0, amt1) - this changes based on swaps, but doesnt matter, bacause liquidity always constant
#     # (decimal0, decimal1, feetier, mint_tick) - fixeds
    
#     mint_liquidity = liq_amounts.get_liquidity(mint_tick, lower_tick_,upper_tick_, amt0_, amt1_, decimal0_, decimal1_)
#     L_you_ = mint_liquidity # always constant, according to JNP
    
#     inRange_ = check_inRange(ctick_, lower_tick_,upper_tick_)
    
#     if inRange_:
#         pool_share_ = (L_you_/L_Pool_)
#         pool_fee_rate_ = float(feeTier) / 10000
#         fees_0 = pool_share_ * amount0_p * pool_fee_rate_ / 100
#         fees_1 = pool_share_ * amount1_p * pool_fee_rate_ / 100
#     else:
#         fees_0 = 0
#         fees_1 = 0
#         pool_share_ = 0
    
#     return fees_0, fees_1, pool_share_, L_you_

# def calculate_pnl_from_positions(df_merged, df_positions):
#     # TODO: re-calculate amount0 and amount1 to account for every swaps, not just deltas of that hour
#     # use amount0_new and amount1_new instead of amountUSD, and get total accured fees in tokens
#     # Swap_amountIn > so the fees is get from the (+) positive tokens
#     # Therefore, fees you get is in 
#     # Convert those tokens into value (convert using TODAY's value or mints's value?)
    
#     merge_feat = ['periodStartUnix', 'tick','liquidity', 'amountUSD', 'amount0_p', 'amount1_p']

#     df_positions_fees = df_positions.merge(df_merged[merge_feat], how='left', on='periodStartUnix')
#     df_positions_fees['liquidity'] = df_positions_fees['liquidity'].astype(float)

#     df_positions_fees = df_positions_fees.reset_index() # cleaning it just in case
#     for index, row in df_positions_fees.iterrows():
#         row = row.copy()
#         fees_0, fees_1, pool_share, L_you = get_fees(row['lower_tick'],row['upper_tick'],row['tick'],
#                         float(row['liquidity']),row['amountUSD'],
#                         row['amount0_p'], row['amount1_p'],
#                         MINT_AMT0, MINT_AMT1, DECIMAL0, DECIMAL1, FEETIER, MINT_TICK)
       
#         df_positions_fees.loc[index, 'fees_0'] = fees_0
#         df_positions_fees.loc[index, 'fees_1'] = fees_1
#         df_positions_fees.loc[index, 'pool_share'] = pool_share
#         df_positions_fees.loc[index, 'L_you'] = L_you
        
#     return df_positions_fees

In [56]:
def get_fees_from_dynamic_positions(df_merged_, df_positions_):
    
    merge_feat_ = ['periodStartUnix', 'tick','liquidity','sqrtPrice', 
                   'amountUSD', 'amount0_p', 'amount1_p']
    df_ = df_positions_.merge(df_merged_[merge_feat_], how='left',on='periodStartUnix')
    df_['liquidity'] = df_['liquidity'].astype(float) # as float
    df_['sqrtPrice'] = df_['sqrtPrice'].astype(float)
#     df_['feeGrowthGlobal0X128'] = df_['feeGrowthGlobal0X128'].astype(float)
#     df_['feeGrowthGlobal1X128'] = df_['feeGrowthGlobal1X128'].astype(float)
    
    df_['fees0'] = 0
    df_['fees1'] = 0
    
    # CAREFUL: 
    print('Shape Before dropNA: {}'.format(df_.shape))
    df_ = df_.dropna(subset=['tick','liquidity'])
    print('Shape After dropNA: {}'.format(df_.shape))
    
    for idx in range(int(df_.iloc[0]['block_amount'])):
        # assigns features neccessary for fees calculation
        df_['L_you'] = df_.apply(lambda x: liq_amounts.get_liquidity(x['mint_tick'], x['lower_tick_'+str(idx)],
                                                x['upper_tick_'+str(idx)], x['mint_amount0_'+str(idx)], x['mint_amount1_'+str(idx)],
                                                x['decimal0'], x['decimal1']), axis=1)

    #     df_['pool_share'] = df_['L_you']/df_['liquidity']
        df_['pool_share'] = df_['L_you'] / ( df_['liquidity'] + df_['L_you'] )
        df_['inRange'] = df_.apply(lambda x: check_inRange(x['tick'], 
                                                x['lower_tick_'+str(idx)], x['upper_tick_'+str(idx)]),axis=1)

        # Calculate Fees via amount_p
        df_['fees0'] += df_['inRange'] * df_['pool_share'] * df_['amount0_p'] * df_['feeTier'] / (10000*100)
        df_['fees1'] += df_['inRange'] * df_['pool_share'] * df_['amount1_p'] * df_['feeTier'] / (10000*100)

    # Calculate Fees via feeGrowthGlobal0X128_hour
#     decimal0 = df_['decimal0']
#     decimal1 = df_['decimal1'] # / (10**decimal0) 
#     df_['fees0_G'] = df_['inRange'] * ( df_['feeGrowthGlobal0X128_hour'] * df_['L_you'] )  / (2**128) / (10**decimal0)  
#     df_['fees1_G'] = df_['inRange'] * ( df_['feeGrowthGlobal1X128_hour'] * df_['L_you'] ) / (2**128) / (10**decimal1) 
    
    return df_

In [60]:
def get_fees_from_positions(df_merged_, df_positions_):
    
    merge_feat_ = ['periodStartUnix', 'tick','liquidity','sqrtPrice', 
                   'amountUSD', 'amount0_p', 'amount1_p']
    df_ = df_positions_.merge(df_merged_[merge_feat_], how='left',on='periodStartUnix')
    df_['liquidity'] = df_['liquidity'].astype(float) # as float
    df_['sqrtPrice'] = df_['sqrtPrice'].astype(float)
#     df_['feeGrowthGlobal0X128'] = df_['feeGrowthGlobal0X128'].astype(float)
#     df_['feeGrowthGlobal1X128'] = df_['feeGrowthGlobal1X128'].astype(float)
    
    
    # CAREFUL: 
    print('Shape Before dropNA: {}'.format(df_.shape))
    df_ = df_.dropna(subset=['tick','liquidity'])
    print('Shape After dropNA: {}'.format(df_.shape))
    
    # assigns features neccessary for fees calculation
    df_['L_you'] = df_.apply(lambda x: liq_amounts.get_liquidity(x['mint_tick'], x['lower_tick'],
                                            x['upper_tick'], x['mint_amount0'], x['mint_amount1'],
                                            x['decimal0'], x['decimal1']), axis=1)

#     df_['pool_share'] = df_['L_you']/df_['liquidity']
    df_['pool_share'] = df_['L_you'] / ( df_['liquidity'] + df_['L_you'] )
    df_['inRange'] = df_.apply(lambda x: check_inRange(x['tick'], 
                                            x['lower_tick'], x['upper_tick']),axis=1)

    # Calculate Fees via amount_p
    df_['fees0'] = df_['inRange'] * df_['pool_share'] * df_['amount0_p'] * df_['feeTier'] / (10000*100)
    df_['fees1'] = df_['inRange'] * df_['pool_share'] * df_['amount1_p'] * df_['feeTier'] / (10000*100)

    # Calculate Fees via feeGrowthGlobal0X128_hour
#     decimal0 = df_['decimal0']
#     decimal1 = df_['decimal1'] # / (10**decimal0) 
#     df_['fees0_G'] = df_['inRange'] * ( df_['feeGrowthGlobal0X128_hour'] * df_['L_you'] )  / (2**128) / (10**decimal0)  
#     df_['fees1_G'] = df_['inRange'] * ( df_['feeGrowthGlobal1X128_hour'] * df_['L_you'] ) / (2**128) / (10**decimal1) 
    
    return df_

### Main

In [9]:
# # STEP 0:  PROVIDE BY USERS
# UPPER = 1/1844.6164 # 1/1844.6164  # token1 per token0 (in terms of price)
# LOWER = 1/2858.3641 # 1/2858.3641  # 1/2376.6075 
# FEETIER = '3000' # 0.3 = 3000/10000
# TOKEN0 = 'USDC'
# TOKEN1 = 'WETH'
# # needs to start with the hour (unixTime % 3600 = 0)
# MINT_TIME = 1622304000 # 4PM May 29 (UTC/GMT)
# BURN_TIME = 1627866000 # 1AM Aug 02 (UTC/GMT)

# # STEP 1 (TODO): CALCULATED BY ALGORITHM, JNP's amounts_relation ?
# # THIS CHANGES over time when swaps happens, and we need this to calculate IL
# MINT_AMT0 = 252208.7053
# MINT_AMT1 = 99.0604

In [10]:
# # Get required constants
# DECIMAL0 = get_decimals(TOKEN0)
# DECIMAL1 = get_decimals(TOKEN1)
# print(DECIMAL0, DECIMAL1)

# MINT_PRICE, MINT_TICK = get_cprice_tick_at_mint(TOKEN0, TOKEN1, FEETIER, MINT_TIME, DECIMAL0, DECIMAL1)
# print(MINT_PRICE, 1/MINT_PRICE)
# print(MINT_TICK)

In [11]:
# # STEP 3 (TODO): Generate df_positions
# df_positions = get_positions_fixed_strategy(MINT_TIME, BURN_TIME, UPPER, LOWER, 
#                                             DECIMAL0, DECIMAL1, FEETIER, MINT_AMT0,
#                                            MINT_AMT1, MINT_TICK)

In [12]:
# # Get DF_Merged (accumulated swaps data)
# # df_merged = pd.read_csv('../data/df_merged_tmp_USDC-WETH-3000-timestamp-1627873200-1620169200.csv')
# df_merged = load_data.get_data(TOKEN0, TOKEN1, FEETIER)

In [13]:
# # STEP 4 (TODO): Calculate Fees, IL and PNL at each hour, and append to df_positions
# df_positions_fees = calculate_pnl_from_positions(df_merged, df_positions)

In [14]:
# STEP 5 (TODO): also accoutning for minting/burning/swaping costs into PNL

In [15]:
# TOKEN0_TO_USD = 1
# TOKEN1_TO_USD = 2561.94

# fees_token0 = df_positions_fees['fees_0'].sum()
# fees_token1 = df_positions_fees['fees_1'].sum()

# totalfees_USD = ( fees_token0*TOKEN0_TO_USD )+ (fees_token1 *TOKEN1_TO_USD)

# print(fees_token0, fees_token1, totalfees_USD)
# revert_fees0 = 37795.612553
# revert_fees1 = 16.845536
# error_margin_fees0 = (1 - fees_token0/revert_fees0)*100
# error_margin_fees1 = (1 - fees_token1/revert_fees1)*100
# print('error margin token0 = {} %, token1 = {} %'.format(error_margin_fees0, error_margin_fees1))

### Testing margin of Errors

In [16]:
# References: 
# 1. https://revert.finance/#/uniswap-position/27782
# 2. https://revert.finance/#/uniswap-position/2067
# 3. https://revert.finance/#/uniswap-position/19724

# 4. https://revert.finance/#/uniswap-position/27783
# 5. https://revert.finance/#/uniswap-position/27784 <very narrow band>
# 6. https://revert.finance/#/uniswap-position/27785 <short time frame>
# 7. https://revert.finance/#/uniswap-position/27786 <short time frame>
# 8. https://revert.finance/#/uniswap-position/27788 <XOR/WETH same price>
# 9. https://revert.finance/#/uniswap-position/27789 <very wide band>

UPPERS = [1/1844.6164 , 1.0014, 1/1655.7876, 1/79.8206, 1/98.4718, 
          10.8361, 1/84.2491, 0.1449, 1/10.0130]
LOWERS = [1/2858.3641 , 1.0004, 1/3334.2300, 1/109.7017, 1/99.6605, 
          19.9823, 1/99.0644, 6.9026, 1/249.5662] 
FEETIERS = ['3000' , '500', '10000', '3000', '3000', 
            '3000', '3000', '3000', '3000']
TOKEN0S = ['USDC' , 'DAI', 'USDC', 'UNI', 'UNI', 
           'WBTC', 'LINK', 'XOR', 'UNI']
TOKEN1S = ['WETH' , 'USDC', 'WETH', 'WETH', 'WETH', 
           'WETH', 'WETH', 'WETH', 'WETH']

# _____ ,may6th, may22nd
MINT_TIMES = [1622304000, 1621720800, 1621641600, 1622322000, 1622322000, 
             1622325600, 1622325600, 1622325600, 1622325600] 
# NOW_1, NOW_1, NOW_2, 
BURN_TIMES = [1627902000, 1627902000, 1627894800, 1624453200, 1627927200, 
             1623153600, 1622548800, 1627927200, 1627927200] 

MINT_AMOUNT0S = [252208.7053 , 13068.6711, 771.4900, 400,    77.1286,
                0.3124, 2256.3660, 0.4500, 199.7828]
MINT_AMOUNT1S = [99.0604 ,     12194.3375, 0.2697,   1.9279,  0.5300,
                4.6689, 23.3172,    0.0001, 1.0883]

# TOKEN0_TO_USDS = [1, 1, 1 ]
# TOKEN1_TO_USDS = [2578.84 , 1, 2578.84 ]

REVERT_FEES0 = [37795.612553 , 421.148243, 136.562165,   5.178524, 1.582686, 
               0.001442, 23.447256, 0.000896, 1.521691]
REVERT_FEES1 = [16.845536 ,    420.589287, 0.060010 ,   0.044819, 0.014380,
               0.015157, 0.315429,  0.000137, 0.013011]

In [17]:
for i in range(5,9):
    print(i)

5
6
7
8


In [66]:
def get_positions_dynamic_strategy(df_merged, capital_per_block, block_amount, block_size, block_spacing, shift_theta, decimal0_, decimal1_, feetier_):

    pos = BlockPosition(df_merged.iloc[0]['tick'], block_size, block_spacing, block_amount)
    pos_array = []
    
    prev_row = df_merged.iloc[0]

    for index, row in df_merged.iterrows():

        curr_tick = prev_row.tick
        # if(put_in_price_tick != 0):
        #     print(hourly_fee * put_in_price_tick/tvl_per_tick)
        optimized = False
        changed = False

        if(abs(pos.center_tick - curr_tick) > pos.block_amount * pos.block_spacing):
            pos.center_tick = curr_tick
            changed = True
            pos.block_array = pos.gen_block_array()
        else:
            while(True):
                if(abs(pos.center_tick - curr_tick) >= shift_theta):
                    changed = True
                    if(pos.center_tick > curr_tick):
                        pos.shift_left()
                    else:
                        pos.shift_right()
                else:
                    break

        array_item = {}
                
        for idx, block in enumerate(pos.block_array):
            array_item['index_'+str(idx)] = block['index']
            array_item['lower_tick_'+str(idx)] = block['lower_tick']
            array_item['upper_tick_'+str(idx)] = block['upper_tick']
            amount0, amount1 = get_amounts(convert_token0_to_1(prev_row['tick'], capital_per_block), prev_row['tick'], block['lower_tick'], block['upper_tick'])
            array_item['mint_amount0_' + str(idx)] = amount0
            array_item['mint_amount1_' + str(idx)] = amount1
            
        array_item['periodStartUnix'] = prev_row['periodStartUnix']
        array_item['decimal0'] = decimal0_
        array_item['decimal1'] = decimal1_
        array_item['feeTier'] = int(feetier_)    
        array_item['mint_tick'] = int(prev_row['tick'])
        array_item['block_amount'] = block_amount
        
        pos_array.append(array_item)
        
        prev_row = row

    pos_df = pd.DataFrame(pos_array)

    return pos_df

UPPER = 1/1844.6164  # token1 per token0 (in terms of price)
LOWER = 1/2858.3641  # 1/2376.6075 
FEETIER = '3000' # 0.3 = 3000/10000
TOKEN0 = 'DAI'
TOKEN1 = 'WETH'
DECIMAL0 = get_decimals(TOKEN0)
DECIMAL1 = get_decimals(TOKEN1)

BLOCK_AMOUNT = 5
BLOCK_SPACING = 120
BLOCK_SIZE = 180
SHIFT_THETA = 720
STARTING_CAPITAL = 500000
CAPITAL_PER_BLOCK = STARTING_CAPITAL/BLOCK_AMOUNT

df_positions = get_positions_dynamic_strategy(df_merged, CAPITAL_PER_BLOCK, BLOCK_AMOUNT, BLOCK_SIZE, BLOCK_SPACING, SHIFT_THETA, DECIMAL0, DECIMAL1, FEETIER)

df_positions_fees = get_fees_from_dynamic_positions(df_merged, df_positions)
df_positions_fees.to_csv('../data_fees/test_results_{}.csv'.format(i))

# # Fees Summary
# revert_fees0 = REVERT_FEES0[i]
# revert_fees1 = REVERT_FEES1[i]

fees_token0 = df_positions_fees['fees0'].sum()
fees_token1 = df_positions_fees['fees1'].sum()

# fees_token0_G = df_positions_fees['fees0_G'].sum()
# fees_token1_G = df_positions_fees['fees1_G'].sum()


# print('Revert.finance fees: ')
# print(revert_fees0, revert_fees1)

print(' ')
print('Fees from amount_p:')
print(fees_token0, fees_token1)
print('Costs from shifting positions')
print(df_positions.iloc[-1]['index_0'] * 30 + df_positions.iloc[-1]['index_0']*0.003 * CAPITAL_PER_BLOCK)


 
get_token_id: DAI
{'data': {'tokens': [{'id': '0x6b175474e89094c44da98b954eedeac495271d0f', 'name': 'Dai Stablecoin', 'symbol': 'DAI'}]}}
 
get_token_id: WETH
{'data': {'tokens': [{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'name': 'Wrapped Ether', 'symbol': 'WETH'}]}}

 get_pool_id for feeTier: 3000
{'data': {'pools': [{'feeTier': '3000', 'id': '0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8', 'token0': {'symbol': 'DAI'}, 'token1': {'symbol': 'WETH'}}]}}
Shape Before dropNA: (2184, 39)
Shape After dropNA: (2184, 39)
 
Fees from amount_p:
414188.8077579841 161.77195503648716
Costs from shifting positions
142230.0


In [36]:
df_positions

,index_0,lower_tick_0,upper_tick_0,mint_amount0_0,mint_amount1_0,index_1,lower_tick_1,upper_tick_1,mint_amount0_1,mint_amount1_1,...,lower_tick_3,upper_tick_3,mint_amount0_3,mint_amount1_3,index_4,lower_tick_4,upper_tick_4,mint_amount0_4,mint_amount1_4,periodStartUnix
0,1,-81684,-81505,0,296.639234,2,-81504,-81325,0.0,296.639234,...,-81144,-80965,1000000.0,0.0,5,-80964,-80785,1000000.0,0,1620158400
1,1,-81684,-81505,0,296.639234,2,-81504,-81325,0.0,296.639234,...,-81144,-80965,1000000.0,0.0,5,-80964,-80785,1000000.0,0,1620162000
2,1,-81684,-81505,0,296.639234,2,-81504,-81325,0.0,296.639234,...,-81144,-80965,1000000.0,0.0,5,-80964,-80785,1000000.0,0,1620165600
3,1,-81684,-81505,0,296.639234,2,-81504,-81325,0.0,296.639234,...,-81144,-80965,1000000.0,0.0,5,-80964,-80785,1000000.0,0,1620169200
4,3,-81324,-81145,0,305.733923,4,-81144,-80965,0.0,305.733923,...,-80784,-80605,1000000.0,0.0,7,-80604,-80425,1000000.0,0,1620172800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2179,873,-78735,-78556,0,399.335469,852,-78555,-78376,0.0,399.335469,...,-78195,-78016,1000000.0,0.0,867,-78015,-77836,1000000.0,0,1628053200
2180,873,-78735,-78556,0,401.457459,852,-78555,-78376,0.0,401.457459,...,-78195,-78016,1000000.0,0.0,867,-78015,-77836,1000000.0,0,1628056800
2181,852,-78555,-78376,0,403.388990,865,-78375,-78196,0.0,403.388990,...,-78015,-77836,1000000.0,0.0,874,-77835,-77656,1000000.0,0,1628060400
2182,852,-78555,-78376,0,404.681837,865,-78375,-78196,0.0,404.681837,...,-78015,-77836,1000000.0,0.0,874,-77835,-77656,1000000.0,0,1628064000


In [61]:
print(DECIMAL0, DECIMAL1)
print(MINT_PRICE, 1/MINT_PRICE)
print(MINT_TICK)

# Generate data
df_positions = get_positions_fixed_strategy(MINT_TIME, BURN_TIME, UPPER, LOWER, 
                                            DECIMAL0, DECIMAL1, FEETIER, MINT_AMT0,
                                           MINT_AMT1, MINT_TICK)
# Load merged data
df_merged = pd.read_csv('../data/df_merged_tmp_DAI-WETH-3000-timestamp-1627574400-1620169200.csv')

# Fees Calculations
df_positions_fees = get_fees_from_positions(df_merged, df_positions)
df_positions_fees.to_csv('../data_fees/test_results_{}.csv'.format(i))

# # Fees Summary
# revert_fees0 = REVERT_FEES0[i]
# revert_fees1 = REVERT_FEES1[i]

fees_token0 = df_positions_fees['fees0'].sum()
fees_token1 = df_positions_fees['fees1'].sum()

# fees_token0_G = df_positions_fees['fees0_G'].sum()
# fees_token1_G = df_positions_fees['fees1_G'].sum()


# print('Revert.finance fees: ')
# print(revert_fees0, revert_fees1)

print(' ')
print('Fees from amount_p:')
print(fees_token0, fees_token1)
# print('Fees from feeGrowthGlobal0X128_hour:')
# print(fees_token0_G, fees_token1_G)

# print(' ')
# error_margin_fees0 = (1 - fees_token0/revert_fees0)*100
# error_margin_fees1 = (1 - fees_token1/revert_fees1)*100
# print('(amount) error margin token0 = {} %, token1 = {} %'.format(error_margin_fees0, error_margin_fees1))

# error_margin_fees0_G = (1 - fees_token0_G/revert_fees0)*100
# error_margin_fees1_G = (1 - fees_token1_G/revert_fees1)*100
# print('(feeGrowthGlobal) error margin token0 = {} %, token1 = {} %'.format(error_margin_fees0_G, error_margin_fees1_G))

18.0 18.0
0.00041721415329092044 2396.850615234779
-77823
Shape Before dropNA: (1442, 17)
Shape After dropNA: (1436, 17)
 
Fees from amount_p:
27635.526287731413 12.457597342268532


In [38]:
for i in range(3,10):
    # SETUPS
    UPPER = UPPERS[i]
    LOWER = LOWERS[i] 
    FEETIER = FEETIERS[i]
    TOKEN0 = TOKEN0S[i]
    TOKEN1 = TOKEN1S[i]
    MINT_TIME = MINT_TIMES[i] 
    BURN_TIME = BURN_TIMES[i] 
    MINT_AMT0 = MINT_AMOUNT0S[i]
    MINT_AMT1 = MINT_AMOUNT1S[i]

    # Get required constants
    DECIMAL0 = get_decimals(TOKEN0)
    DECIMAL1 = get_decimals(TOKEN1)
    MINT_PRICE, MINT_TICK = get_cprice_tick_at_mint(TOKEN0, TOKEN1, FEETIER, MINT_TIME, DECIMAL0, DECIMAL1)
    print(DECIMAL0, DECIMAL1)
    print(MINT_PRICE, 1/MINT_PRICE)
    print(MINT_TICK)

    # Generate data
    df_positions = get_positions_fixed_strategy(MINT_TIME, BURN_TIME, UPPER, LOWER, 
                                                DECIMAL0, DECIMAL1, FEETIER, MINT_AMT0,
                                               MINT_AMT1, MINT_TICK)
    # Load merged data
    df_merged = load_data.get_data(TOKEN0, TOKEN1, FEETIER)

    # Fees Calculations
    df_positions_fees = get_fees_from_positions(df_merged, df_positions)
    df_positions_fees.to_csv('../data_fees/test_results_{}.csv'.format(i))

    # Fees Summary
    revert_fees0 = REVERT_FEES0[i]
    revert_fees1 = REVERT_FEES1[i]

    fees_token0 = df_positions_fees['fees0'].sum()
    fees_token1 = df_positions_fees['fees1'].sum()

    fees_token0_G = df_positions_fees['fees0_G'].sum()
    fees_token1_G = df_positions_fees['fees1_G'].sum()


    print('Revert.finance fees: ')
    print(revert_fees0, revert_fees1)

    print(' ')
    print('Fees from amount_p:')
    print(fees_token0, fees_token1)
    print('Fees from feeGrowthGlobal0X128_hour:')
    print(fees_token0_G, fees_token1_G)

    print(' ')
    error_margin_fees0 = (1 - fees_token0/revert_fees0)*100
    error_margin_fees1 = (1 - fees_token1/revert_fees1)*100
    print('(amount) error margin token0 = {} %, token1 = {} %'.format(error_margin_fees0, error_margin_fees1))

    error_margin_fees0_G = (1 - fees_token0_G/revert_fees0)*100
    error_margin_fees1_G = (1 - fees_token1_G/revert_fees1)*100
    print('(feeGrowthGlobal) error margin token0 = {} %, token1 = {} %'.format(error_margin_fees0_G, error_margin_fees1_G))

 
get_token_id: UNI
{'data': {'tokens': [{'id': '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984', 'name': 'Uniswap', 'symbol': 'UNI'}]}}
 
get_token_id: WETH
{'data': {'tokens': [{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'name': 'Wrapped Ether', 'symbol': 'WETH'}]}}

 get_pool_id for feeTier: 3000
{'data': {'pools': [{'feeTier': '3000', 'id': '0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801', 'token0': {'symbol': 'UNI'}, 'token1': {'symbol': 'WETH'}}]}}
18.0 18.0
0.01003506398733612 99.65058531385182
-46019
 
get_token_id: UNI
{'data': {'tokens': [{'id': '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984', 'name': 'Uniswap', 'symbol': 'UNI'}]}}
 
get_token_id: WETH
{'data': {'tokens': [{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'name': 'Wrapped Ether', 'symbol': 'WETH'}]}}

 get_pool_id for feeTier: 3000
{'data': {'pools': [{'feeTier': '3000', 'id': '0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801', 'token0': {'symbol': 'UNI'}, 'token1': {'symbol': 'WETH'}}]}}
 

 Queried poolHourDatas

KeyboardInterrupt: 

In [ ]:
# # df_positions_fees = get_fees_from_positions(df_merged, df_positions)

# fees_token0 = df_positions_fees['fees0'].sum()
# fees_token1 = df_positions_fees['fees1'].sum()

# totalfees_USD = ( fees_token0*TOKEN0_TO_USD )+ (fees_token1 *TOKEN1_TO_USD)

# print(fees_token0, fees_token1, totalfees_USD)
# print(revert_fees0,revert_fees1)

# error_margin_fees0 = (1 - fees_token0/revert_fees0)*100
# error_margin_fees1 = (1 - fees_token1/revert_fees1)*100
# print('error margin token0 = {} %, token1 = {} %'.format(error_margin_fees0, error_margin_fees1))

In [ ]:
# 35835.9997570211 16.065701314551056 77266.87293503794
# error margin token0 = 5.184762631458817 %, token1 = 4.629325451258682 %


# 167.380859180713 0.07378978912937459 357.67291897910934
# error margin token0 = -22.567520206429805 %, token1 = -22.9624881342686 %